### Selected model is noaug-grad, here are the results.

In [12]:
from torch import nn
import pretrainedmodels
import pandas as pd
import torch
import numpy as np

In [13]:
#load efficient net model
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')
model._dropout = nn.Sequential(nn.Dropout(p=0.5))
model._fc = nn.Sequential(nn.Linear(1280,3),nn.Softmax())
#add last layer with 3 classes

#put model on gpu
model.cuda()
#load trained model with weights
model.load_state_dict(torch.load('torch_updated_trained_model.pt'))

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [14]:
#create empty pandas data frame with specified colunm names.
results = pd.DataFrame(columns=['Id','task_1','task_2'])


In [15]:
#load test images
from torchvision import datasets
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
#resize,crop,transfer to pytorch tensor and normalize the images
transform = transforms.Compose([transforms.Resize(299),
                                     transforms.CenterCrop(299),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])                                        
test_data = datasets.ImageFolder(root = 'data/test',transform = transform)
test_loader = DataLoader(test_data,batch_size = 1,shuffle = False,num_workers = 0)
print(test_data.class_to_idx)

{'melanoma': 0, 'nevus': 1, 'seborrheic_keratosis': 2}


In [16]:
#get all pictures id's
import glob
test_dir = glob.glob('data/*/*/*')
#add to pandas dataframe
results['Id'] = test_dir
print(results)

Id task_1 task_2
0                data\test\melanoma\ISIC_0012258.jpg    NaN    NaN
1                data\test\melanoma\ISIC_0012356.jpg    NaN    NaN
2                data\test\melanoma\ISIC_0012369.jpg    NaN    NaN
3                data\test\melanoma\ISIC_0012395.jpg    NaN    NaN
4                data\test\melanoma\ISIC_0012425.jpg    NaN    NaN
..                                               ...    ...    ...
595  data\test\seborrheic_keratosis\ISIC_0014647.jpg    NaN    NaN
596  data\test\seborrheic_keratosis\ISIC_0014648.jpg    NaN    NaN
597  data\test\seborrheic_keratosis\ISIC_0014649.jpg    NaN    NaN
598  data\test\seborrheic_keratosis\ISIC_0014652.jpg    NaN    NaN
599  data\test\seborrheic_keratosis\ISIC_0014653.jpg    NaN    NaN

[600 rows x 3 columns]


In [17]:
#define criterion
criterion = nn.CrossEntropyLoss()


In [18]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#make predictions
task_1 = []
task_2 = []
def test(loaders, model, criterion, use_cuda):
    #make list probalities for melanoma (task_1) and seborrheic keratosis (task_2)
    
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders):
        with torch.no_grad():
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #extract melanoma and seborrheic_keratosis_proba probabilities
            melanoma_proba = output[0].tolist()[0]
            seborrheic_keratosis_proba = output[0].tolist()[1]
            
            #append melanoma probablilies
            task_1.append(melanoma_proba)
            #append seborrheic_keratosis probablilies
            task_2.append(seborrheic_keratosis_proba)
       

            

            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(test_loader, model, criterion, True)

Test Loss: 0.822462


Test Accuracy: 76% (456/600)


In [19]:
#add those lists as entries to our task_1 and task_2 columns

results['task_1'] = task_1
results['task_2'] = task_2
print(results)

Id    task_1    task_2
0                data\test\melanoma\ISIC_0012258.jpg  0.218362  0.404887
1                data\test\melanoma\ISIC_0012356.jpg  0.388765  0.554406
2                data\test\melanoma\ISIC_0012369.jpg  0.810520  0.102277
3                data\test\melanoma\ISIC_0012395.jpg  0.429009  0.568962
4                data\test\melanoma\ISIC_0012425.jpg  0.201353  0.575334
..                                               ...       ...       ...
595  data\test\seborrheic_keratosis\ISIC_0014647.jpg  0.050430  0.297376
596  data\test\seborrheic_keratosis\ISIC_0014648.jpg  0.000185  0.003070
597  data\test\seborrheic_keratosis\ISIC_0014649.jpg  0.002220  0.022591
598  data\test\seborrheic_keratosis\ISIC_0014652.jpg  0.001895  0.010487
599  data\test\seborrheic_keratosis\ISIC_0014653.jpg  0.000208  0.003328

[600 rows x 3 columns]


In [20]:
#convert the pandas data frame to a csv
results.to_csv('predictions_efficient_net.csv')

# Results
The model performed well at classifying melanoma and bening lesions correctly. These results where achieved by setting the melanoma threshold to a 0.3 (as in if the model thinks theres a 30% chance its melanoma it is classified as melanoma.) This is done to make sure that we are being extra carefuly in missing a diagnsosis, as it is preferable to misdiagnose someone with melanoma then to miss the diagnsosis completly. The overall accuracy of the model is 76%. This model achieves a higher rate of melanoma diagnosis then dermatologists (66% vs 71%).

<img src="eff_net_0.3_confusion.png" alt="eff_net_confusion" width="500"/>          <img src="eff_net_base_roc.png" alt="eff_net roc" width="500"/>                               
